# INTRODUÇÃO

- **myhome.com**: `O produto principal da empresa é uma plataforma de compra e venda de imóveis com curadoria automatizada por Machine Learning` que se adapta ao que tanto os vendedores quanto compradores procuram. A empresa é focada em imóveis para fins residenciais e o público em outros países gira em torno de pessoas que estão adquirindo o primeiro imóvel próprio.  A empresa está se preparando para entrar no mercado brasileiro, a começar pela cidade de São Paulo. A `myhome.com ainda está entendendo o ambiente do mercado brasileiro` e não tem especialistas no assunto quando se trata do Brasil ou da cidade de São Paulo.



## Infos & Objetivos

gostaríamos de entender melhor o cenário do mercado na cidade. 
- Qual o perfil dos imóveis disponíveis para venda?
- Onde seria o lugar ideal para começar a prospectar clientes? 
- `Uma das coisas que queriamos entender é se há maior disponibilidade de imoveis em regioes da zona sul da cidade comparado com outras regioes? Se regioes mais afastadas do centro Possuem maior disponibilidade ou nao`
  
O time de Engenharia de Dados disponibilizou um conjunto de dados no Data Lake da empresa que contém um `dump` de listings do mercado imobiliário de São Paulo proveniente de várias fontes.

**Contact with business**
- Tem algo além dessas duas questões que vocÊs queiram validar?
- é possível disponibilizar o dump em algum outro ambiente?

# IDEIAS E OBSERVAÇÕES

- processamento das colunas `address` e `pricingInfos` da listings_df

#INITIALIZING

In [8]:
db_path = '/content/drive/MyDrive/0_Machine_Learning/Projetos/Myhome.com/listings_dump.json'

#LIBS

# New Section

In [12]:
import pandas as pd
import json

# CHECKING THE DATASET

In [10]:
# From version 0.19.0 of Pandas you can use the `lines` parameter,
# Otherwise it rises a `Trailing Error`
listings_df = pd.read_json(db_path, lines=True)

In [11]:
listings_df.head()

,usableAreas,description,title,createdAt,publisherId,unitTypes,listingStatus,id,parkingSpaces,updatedAt,owner,images,address,suites,publicationType,bathrooms,totalAreas,bedrooms,pricingInfos
0,388.0,"04 dorms sendo 01 suíte e closet, sala de esta...",PRÓXIMO A AVENIDA PRESIDENTE TANCREDO NEVES,2017-02-07T13:21:40Z,f4603b2b52,TWO_STORY_HOUSE,ACTIVE,787c7bd19d,6.0,2018-12-06T19:27:12.623Z,False,[https://s3-sa-east-1.amazonaws.com/vr.images....,"{'country': 'BR', 'zipCode': '04290030', 'city...",1.0,STANDARD,3.0,388.0,4.0,"{'price': 700000, 'businessType': 'SALE'}"
1,129.0,"03 dorms sendo 01 suíte, sala, sala de jantar,...",PRÓXIMO A FACULDADE UNIP CAMPUS ANCHIETA,2016-03-21T18:35:17Z,f4603b2b52,HOME,ACTIVE,4d68c0cdbe,2.0,2018-12-12T13:17:23.547Z,False,[https://s3-sa-east-1.amazonaws.com/vr.images....,"{'country': 'BR', 'zipCode': '04187320', 'city...",1.0,STANDARD,2.0,129.0,3.0,"{'price': 336000, 'businessType': 'SALE'}"
2,396.0,"Andar com 395,70m² de área útil, 04 wcs, 05 va...",Excelente Conjunto Comercial na Av. Paulista,2018-12-18T23:47:03.425Z,501f6d5e94,COMMERCIAL_PROPERTY,ACTIVE,e7e0b554ac,5.0,NaN,False,[http://static.nidoimovel.com.br/d3d9446802a44...,"{'country': '', 'zipCode': '01311000', 'city':...",0.0,STANDARD,4.0,NaN,0.0,"{'period': 'MONTHLY', 'yearlyIptu': 4040, 'pri..."
3,80.0,"Viva melhor e de um jeito novo, com privilégio...","Apartamento com 80m², 2 vagas próximo ao Parqu...",2018-10-26T16:18:28.915Z,527c7b0cc0,APARTMENT,ACTIVE,6654d93423,2.0,2018-11-08T16:38:35.434Z,False,[https://vr-prod-vivapro-images.s3.amazonaws.c...,"{'country': '', 'zipCode': '04550004', 'city':...",1.0,STANDARD,2.0,80.0,3.0,"{'yearlyIptu': 1610, 'price': 739643, 'busines..."
4,3322.0,Sua área é ocupada majoritariamente por instit...,"Apartamento 332m², 4 suítes à venda próximo ao...",2018-12-14T18:06:51.342Z,527c7b0cc0,APARTMENT,ACTIVE,9ffaf676ae,5.0,2018-12-14T18:06:55.140Z,False,[https://vr-prod-vivapro-images.s3.amazonaws.c...,"{'country': '', 'zipCode': '04005030', 'city':...",4.0,STANDARD,5.0,332.0,4.0,"{'yearlyIptu': 18900, 'price': 7520099, 'busin..."


será necessária um segundo processamento das colunas `address` e `pricingInfos`, para expandir as infos dos dicts

In [13]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133964 entries, 0 to 133963
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   usableAreas      133211 non-null  float64
 1   description      133964 non-null  object 
 2   title            133964 non-null  object 
 3   createdAt        133964 non-null  object 
 4   publisherId      133964 non-null  object 
 5   unitTypes        133964 non-null  object 
 6   listingStatus    133964 non-null  object 
 7   id               133964 non-null  object 
 8   parkingSpaces    129539 non-null  float64
 9   updatedAt        127367 non-null  object 
 10  owner            133964 non-null  bool   
 11  images           133964 non-null  object 
 12  address          133964 non-null  object 
 13  suites           120347 non-null  float64
 14  publicationType  133964 non-null  object 
 15  bathrooms        133051 non-null  float64
 16  totalAreas       91132 non-null   floa

## Expandindo a coluna `address`

### teste

Primeiro vou fazer um teste de como posso fazer o unpacking dessas infos de forma a criar um dataframe novo com esses atributos

In [16]:
address = listings_df.loc[0,'address']

In [17]:
address

{'city': 'São Paulo',
 'country': 'BR',
 'district': '',
 'geoLocation': {'location': {'lat': -23.612923, 'lon': -46.614222},
  'precision': 'ROOFTOP'},
 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da Saude',
 'neighborhood': 'Jardim da Saúde',
 'state': 'São Paulo',
 'street': 'Rua Juvenal Galeno',
 'streetNumber': '53',
 'unitNumber': '',
 'zipCode': '04290030',
 'zone': 'Zona Sul'}

In [34]:
type(address)

dict

In [38]:
address['geoLocation']['precision']

'ROOFTOP'

In [39]:
address['precision'] = address['geoLocation']['precision']

In [40]:
address

{'city': 'São Paulo',
 'country': 'BR',
 'district': '',
 'geoLocation': {'location': {'lat': -23.612923, 'lon': -46.614222},
  'precision': 'ROOFTOP'},
 'locationId': 'BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da Saude',
 'neighborhood': 'Jardim da Saúde',
 'precision': 'ROOFTOP',
 'state': 'São Paulo',
 'street': 'Rua Juvenal Galeno',
 'streetNumber': '53',
 'unitNumber': '',
 'zipCode': '04290030',
 'zone': 'Zona Sul'}

In [35]:
a = pd.DataFrame(address)
a.head()

,country,zipCode,city,streetNumber,zone,geoLocation,street,locationId,district,unitNumber,state,neighborhood
location,BR,04290030,São Paulo,53,Zona Sul,"{'lon': -46.614222, 'lat': -23.612923}",Rua Juvenal Galeno,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da...,,,São Paulo,Jardim da Saúde
precision,BR,04290030,São Paulo,53,Zona Sul,ROOFTOP,Rua Juvenal Galeno,BR>Sao Paulo>NULL>Sao Paulo>Zona Sul>Jardim da...,,,São Paulo,Jardim da Saúde
